**1. Initialization**

In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals
!pip install -q tensorflow==2.0.0-beta1
!pip install "numpy<1.17"

import numpy as np
import tensorflow as tf

print("TF", tf.__version__)
print("NP", np.__version__)


TF 2.0.0-beta1
NP 1.16.6


**2. Softmax Classification**

In [3]:
X = tf.constant([[1, 2, 1, 1], 
                 [2, 1, 3, 2], 
                 [3, 1, 3, 4], 
                 [4, 1, 5, 5], 
                 [1, 7, 5, 5], 
                 [1, 2, 5, 6],
                 [1, 6, 6, 6],
                 [1, 7, 7, 7]], dtype=tf.float32, shape=[8, 4])    
Y = tf.constant([[0, 0, 1], 
                 [0, 0, 1], 
                 [0, 0, 1], 
                 [0, 1, 0], 
                 [0, 1, 0], 
                 [0, 1, 0],
                 [1, 0, 0],
                 [1, 0, 0]], dtype=tf.float32, shape=[8, 3])

class_num = 3    

W = tf.Variable(tf.random.uniform(shape=[4, class_num], name='weight', minval=0, maxval=1, dtype=tf.float32, seed=3))
b = tf.Variable(tf.random.uniform(shape=[class_num], name='bias', minval=0, maxval=1, dtype=tf.float32, seed=3))

def hypothesis(x, w, b):
    return tf.nn.softmax(tf.linalg.matmul(x, w) + b)

def cost(x, w, b, y):
    H = hypothesis(x, w, b)
    return tf.reduce_mean(-tf.reduce_sum(y * tf.math.log(H), axis=1))

optimizer = tf.keras.optimizers.SGD(learning_rate=1e-1)
loss = lambda: cost(X, W, b, Y) 

for step in range(2001):
    optimizer.minimize(loss, var_list=[W, b])
    if step % 200 == 0:
        print(step, cost(X, W, b, Y))

0 tf.Tensor(1.215736, shape=(), dtype=float32)
200 tf.Tensor(0.5495157, shape=(), dtype=float32)
400 tf.Tensor(0.45369452, shape=(), dtype=float32)
600 tf.Tensor(0.377613, shape=(), dtype=float32)
800 tf.Tensor(0.3049659, shape=(), dtype=float32)
1000 tf.Tensor(0.24294528, shape=(), dtype=float32)
1200 tf.Tensor(0.21946111, shape=(), dtype=float32)
1400 tf.Tensor(0.20016202, shape=(), dtype=float32)
1600 tf.Tensor(0.18387848, shape=(), dtype=float32)
1800 tf.Tensor(0.1699591, shape=(), dtype=float32)
2000 tf.Tensor(0.15793051, shape=(), dtype=float32)


In [4]:
def test(x, w, b):
    y_hat = hypothesis(x, w, b)
    print("y_hat : {}".format(y_hat))
    print("result : {}".format(tf.math.argmax(y_hat, axis=1)))

test(tf.constant([1, 11, 7, 9], dtype=tf.float32, shape=[1, 4]), W, b)

y_hat : [[1.0120906e-02 9.8987013e-01 8.9907326e-06]]
result : [1]


**3. Fancy Softmax Classification**

In [5]:
from google.colab import drive
drive.mount('/gdrive')

!ls /gdrive/My\ Drive/Colab\ Notebooks/data

xy = np.loadtxt('/gdrive/My Drive/Colab Notebooks/data/data-04-zoo.csv', delimiter=',', dtype=np.float32)
x_data = xy[:, 0:-1]
y_data= xy[:, [-1]]

# print(x_data.shape, x_data, len(x_data))
# print(y_data.shape, y_data)

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
data-01-test-score.csv	 data-03-diabetes.csv
data-02-stock_daily.csv  data-04-zoo.csv


In [0]:
X = tf.Variable(x_data, validate_shape=False)
Y = tf.Variable(y_data, validate_shape=False)

class_num = 7
Y_one_hot = tf.one_hot(tf.cast(Y, dtype=tf.int32), depth=7)
Y_one_hot = tf.reshape(Y_one_hot, [-1, class_num])

W = tf.Variable(tf.random.uniform(shape=[16, class_num], name='weight', minval=0, maxval=1, dtype=tf.float32, seed=3))
b = tf.Variable(tf.random.uniform(shape=[class_num], name='bias', minval=0, maxval=1, dtype=tf.float32, seed=3))

In [45]:
def logits(x, w, b):
    return tf.linalg.matmul(x, w) + b

def hypothesis(x, w, b):
    return tf.nn.softmax(logits(x, w, b))

def cost(y_hat, y):
    cost_i = tf.nn.softmax_cross_entropy_with_logits(logits=y_hat, labels=y)
    return tf.reduce_mean(cost_i)

optimizer = tf.keras.optimizers.SGD(learning_rate=1e-1)
loss = lambda: cost(logits(X, W, b), Y_one_hot) 

def prediction(h):
    return tf.math.argmax(h, 1)

def correct_prediction(pred, y):
    return tf.equal(pred, tf.math.argmax(y, 1))

def accuracy(correct_pred):
    return tf.reduce_mean(tf.cast(correct_pred, tf.float32))

for step in range(2001):
    optimizer.minimize(loss, var_list=[W, b])
    if step % 100 == 0:
        hypo = hypothesis(X, W, b)
        pred = prediction(hypo)
        correct_pred = correct_prediction(pred, Y_one_hot)
        print("Step: {:5}\tLoss: {:.3f}\tAcc: {:.2f}%".format(step, cost(logits(X, W, b), Y_one_hot), accuracy(correct_pred)))

Step:     0	Loss: 0.020	Acc: 1.00%
Step:   100	Loss: 0.019	Acc: 1.00%
Step:   200	Loss: 0.019	Acc: 1.00%
Step:   300	Loss: 0.019	Acc: 1.00%
Step:   400	Loss: 0.018	Acc: 1.00%
Step:   500	Loss: 0.018	Acc: 1.00%
Step:   600	Loss: 0.018	Acc: 1.00%
Step:   700	Loss: 0.018	Acc: 1.00%
Step:   800	Loss: 0.017	Acc: 1.00%
Step:   900	Loss: 0.017	Acc: 1.00%
Step:  1000	Loss: 0.017	Acc: 1.00%
Step:  1100	Loss: 0.017	Acc: 1.00%
Step:  1200	Loss: 0.017	Acc: 1.00%
Step:  1300	Loss: 0.016	Acc: 1.00%
Step:  1400	Loss: 0.016	Acc: 1.00%
Step:  1500	Loss: 0.016	Acc: 1.00%
Step:  1600	Loss: 0.016	Acc: 1.00%
Step:  1700	Loss: 0.015	Acc: 1.00%
Step:  1800	Loss: 0.015	Acc: 1.00%
Step:  1900	Loss: 0.015	Acc: 1.00%
Step:  2000	Loss: 0.015	Acc: 1.00%


In [46]:
pred = prediction(hypothesis(X, W, b))
for p, y in zip(pred, y_data.flatten()):
    print("[{0}] Prediction: {1} True Y: {2}".format(int(p) == int(y), p, int(y)))

[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 3 True Y: 3
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 3 True Y: 3
[True] Prediction: 3 True Y: 3
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 1 True Y: 1
[True] Prediction: 3 True Y: 3
[True] Prediction: 6 True Y: 6
[True] Prediction: 6 True Y: 6
[True] Prediction: 6 True Y: 6
[True] Prediction: 1 True Y: 1
[True] Prediction: 0 True Y: 0
[True] Prediction: 3 True Y: 3
[True] Prediction: 0 True Y: 0
[True] Prediction: 1 True Y: 1
[True] Prediction: 1 True Y: 1
[True] Prediction: 0 True Y: 0
[True] Prediction: 1 True Y: 1
[True] Prediction: 5 True Y: 5
[True] Prediction: 4 True Y: 4
[True] Prediction: 4 True Y: 4
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 5 True Y: 5
[True] Prediction: 0 True Y: 0
[True] P